# RetFound Test

In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning import Trainer
from neural_network.vmamba import VisualMamba

def test_forward():
    batch_size = 2
    img_size = 224
    num_classes = 10
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = VisualMamba(img_size=img_size, num_classes=num_classes, mask_ratio=0.75).to(device)
    x = torch.randn(batch_size, 3, img_size, img_size, device=device)

    logits = model(x)
    print("Logits shape:", logits.shape)
    assert logits.shape == (batch_size, num_classes)

def test_training_step():
    if not torch.cuda.is_available():
        print("⚠️ Skipping training test: Mamba requires CUDA")
        return

    batch_size = 4
    img_size = 224
    num_classes = 10

    x = torch.randn(16, 3, img_size, img_size, device="cuda")
    y = torch.randint(0, num_classes, (16,), device="cuda")
    dataset = TensorDataset(x, y)
    dataloader = DataLoader(dataset, batch_size=batch_size)

    model = VisualMamba(img_size=img_size, num_classes=num_classes, mask_ratio=0.75).cuda()

    trainer = Trainer(max_epochs=1, accelerator="gpu", devices=1, fast_dev_run=True)
    trainer.fit(model, train_dataloaders=dataloader, val_dataloaders=dataloader)

    print("Training loop ran successfully ✅")

if __name__ == "__main__":
    test_forward()
    test_training_step()


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/andre/code/StudentMAE/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
LOCAL_RANK: 

Logits shape: torch.Size([2, 10])



  | Name        | Type       | Params | Mode 
---------------------------------------------------
0 | model       | Sequential | 26.0 K | train
1 | patch_embed | Conv2d     | 147 K  | train
2 | backbone    | Sequential | 6.0 M  | train
3 | norm        | LayerNorm  | 384    | train
4 | head        | Linear     | 1.9 K  | train
---------------------------------------------------
6.2 M     Trainable params
0         Non-trainable params
6.2 M     Total params
24.850    Total estimated model params size (MB)
176       Modules in train mode
0         Modules in eval mode
/home/andre/code/StudentMAE/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/andre/code/StudentMAE/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/d

Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.87it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.86it/s]
Training loop ran successfully ✅


# Distillation Test

In [ ]:
# main.py

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer

from distillation.dist import EmbeddingDataset, distill_embeddings
from models.retfound import RETFoundClassifier
from models.vmamba import VisualMamba

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

teacher = RETFoundClassifier(
    checkpoint_path="checkpoints/RETFound_cfp_weights.pth"
).to(device)

student = VisualMamba(
    img_size=224,
    patch_size=16,
    in_chans=3,
    embed_dim=192,
    depth=24,
    learning_rate=1e-4
).to(device)

projector = nn.Linear(192, 1024).to(device)

dummy_data = torch.randn(1000, 3, 224, 224)
embed_loader = DataLoader(EmbeddingDataset(dummy_data), batch_size=32, shuffle=True)

# ---------------- phase 1 ----------------
distill_embeddings(
    teacher=teacher,
    student=student,
    projector=projector,
    dataloader=embed_loader,
    device=device,
    epochs=1,
)

torch.save(student.state_dict(), "vmamba_distilled.pth")

# ================================
# phase 2 lightning finetuning
# ================================

# # freeze backbone — only train heads
# for name, p in student.named_parameters():
#     if not name.startswith("heads."):
#         p.requires_grad_(False)

# # your real labeled dataset
# train_loader = DataLoader(real_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(real_val, batch_size=16)

# trainer = Trainer(
#     max_epochs=10,
#     accelerator="gpu" if torch.cuda.is_available() else "cpu",
# )

# trainer.fit(student, train_loader, val_loader)

# torch.save(student.state_dict(), "vmamba_heads_trained.pth")

Missing keys after loading checkpoint: ['fc_norm.weight', 'fc_norm.bias', 'head.weight', 'head.bias']
[Distill] epoch 1  loss=0.996899


# Final model test

In [24]:
import os
import torch
import torch.nn as nn
import pandas as pd
from torchvision import transforms
from torch.utils.data import DataLoader
from config.constants import *
from dataloader.idrid import IDRiDDataset
from models.vmamba_backbone import VisualMamba


MODEL_PATH = "checkpoints/vmamba_final_head_v3_unmasked.pth"
CSV_OUT = "vmamba_idrid_predictions.csv"


class VmambaFinalModel(nn.Module):
    """
    Loads the final backbone + head exactly as stored in
    vmamba_final_head_v3_unmasked.pth
    """

    def __init__(self):
        super().__init__()

        # ----------------------------
        # 1. Rebuild VisualMamba Backbone
        # ----------------------------
        self.backbone = VisualMamba(
            img_size=IMG_SIZE,
            patch_size=PATCH_SIZE,
            in_chans=IN_CHANS,
            embed_dim=VMAMBA_EMBED_DIM,
            depth=VMAMBA_DEPTH,
            mask_ratio=0.0,
            use_cls_token=False,
        )

        # ----------------------------
        # 2. Rebuild Classification Head (must match training)
        # ----------------------------
        self.head = nn.Sequential(
            nn.Linear(self.backbone.embed_dim, 512),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Dropout(0.3),

            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.2),

            nn.Linear(128, NUM_CLASSES)
        )

        # ----------------------------
        # 3. Load Weights
        # ----------------------------
        print(f"Loading checkpoint: {MODEL_PATH}")
        ckpt = torch.load(MODEL_PATH, map_location="cpu")

        if "state_dict" in ckpt:
            state_dict = ckpt["state_dict"]
        else:
            state_dict = ckpt

        backbone_state = {}
        head_state = {}

        for k, v in state_dict.items():
            # backbone.*
            if k.startswith("backbone."):
                backbone_state[k[len("backbone."):]] = v
            # head.*
            elif k.startswith("head."):
                head_state[k[len("head."):]] = v

        self.backbone.load_state_dict(backbone_state)
        self.head.load_state_dict(head_state)

        # Freeze all parameters
        for p in self.parameters():
            p.requires_grad = False

        self.eval()

    def forward(self, x):
        with torch.no_grad():
            feats = self.backbone.forward_features(x)
            logits = self.head(feats)
            probs = torch.softmax(logits, dim=1)
            return probs


def run_prediction():
    # ----------------------------
    # Transform (must match training!)
    # ----------------------------
    tfm = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
    ])

    # ----------------------------
    # Test dataset
    # ----------------------------
    test_ds = IDRiDDataset(IDRID_PATH, split="test", transform=tfm)
    test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)

    # ----------------------------
    # Load model
    # ----------------------------
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = VmambaFinalModel().to(device)

    paths = []
    labels = []
    preds = []
    probs_list = []

    print("Running predictions on IDRiD test set...")

    for img, label, path in test_loader:
        img = img.to(device)

        probs = model(img)
        pred = torch.argmax(probs, dim=1)

        paths.append(path[0])
        labels.append(label.item())
        preds.append(pred.item())
        probs_list.append(probs.cpu().numpy()[0].tolist())

    # ----------------------------
    # Save to CSV
    # ----------------------------
    df = pd.DataFrame({
        "image_path": paths,
        "true_label": labels,
        "pred_label": preds,
    })

    prob_cols = [f"prob_{i}" for i in range(NUM_CLASSES)]
    df_probs = pd.DataFrame(probs_list, columns=prob_cols)

    df = pd.concat([df, df_probs], axis=1)
    df.to_csv(CSV_OUT, index=False)

    print(f"[✓] Saved predictions to {CSV_OUT}")


if __name__ == "__main__":
    run_prediction()

Found 103 images for split=test
Loading checkpoint: checkpoints/vmamba_final_head_v3_unmasked.pth
Running predictions on IDRiD test set...
[✓] Saved predictions to vmamba_idrid_predictions.csv


In [25]:
# diagnose_vmamba.py
import os
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms

from config.constants import IDRID_PATH, IMG_SIZE, CHECKPOINT_DIR, NUM_CLASSES
from dataloader.idrid import IDRiDDataset
from models.vmamba_backbone import VisualMamba
import torch.nn as nn

CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, "vmamba_final_head_v3_unmasked.pth")
BATCH_SIZE = 8
NUM_WORKERS = 4

def load_backbone_and_head(ckpt_path):
    """Reconstruct VisualMamba backbone and head, load weights from checkpoint."""
    ckpt = torch.load(ckpt_path, map_location="cpu")
    if "state_dict" in ckpt:
        state = ckpt["state_dict"]
    else:
        state = ckpt

    # Build backbone
    backbone = VisualMamba(
        img_size=IMG_SIZE,
        patch_size=16 if hasattr(__import__('config.constants'), 'PATCH_SIZE') is False else 16,  # patched default - replace if needed
        in_chans=3,
        embed_dim=getattr(__import__('config.constants'), 'VMAMBA_EMBED_DIM'),
        depth=getattr(__import__('config.constants'), 'VMAMBA_DEPTH'),
        mask_ratio=0.0,
        use_cls_token=False,
    )

    # Build head (must match training)
    head = nn.Sequential(
        nn.Linear(backbone.embed_dim, 512),
        nn.BatchNorm1d(512),
        nn.GELU(),
        nn.Dropout(0.3),

        nn.Linear(512, 128),
        nn.BatchNorm1d(128),
        nn.GELU(),
        nn.Dropout(0.2),

        nn.Linear(128, NUM_CLASSES)
    )

    # split state dict into backbone / head parts
    backbone_state = {}
    head_state = {}
    for k, v in state.items():
        if k.startswith("backbone."):
            backbone_state[k[len("backbone."):]] = v
        elif k.startswith("head."):
            head_state[k[len("head."):]] = v
        # also accept keys without prefixes (older save formats)
        elif k in backbone.state_dict().keys():
            backbone_state[k] = v
        elif k in head.state_dict().keys():
            head_state[k] = v

    # load weights (use strict=False to allow tiny mismatches but report)
    missing_b, unexpected_b = backbone.load_state_dict(backbone_state, strict=False)
    missing_h, unexpected_h = head.load_state_dict(head_state, strict=False)

    print("Backbone load: missing keys:", missing_b, " unexpected:", unexpected_b)
    print("Head load:     missing keys:", missing_h, " unexpected:", unexpected_h)

    return backbone, head

def diagnostics(backbone, head, batch):
    """Run a suite of diagnostics on a batch (x_cpu, y, paths)."""
    x_cpu, y, paths = batch
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    backbone = backbone.to(device)
    head = head.to(device)
    backbone.eval()
    head.eval()

    print("\n=== DIAGNOSTICS ===")
    print("Batch size:", x_cpu.shape)

    # move to device
    x = x_cpu.to(device)

    # 1) input stats
    with torch.no_grad():
        pixel_means = x.view(x.shape[0], -1).mean(dim=1).cpu().numpy()
        pixel_stds  = x.view(x.shape[0], -1).std(dim=1).cpu().numpy()
    print("\n1) Input pixel means (per image):", np.round(pixel_means, 6))
    print("   Input pixel stds  (per image):", np.round(pixel_stds, 6))

    # 2) patch_embed output stats
    with torch.no_grad():
        pe = backbone.patch_embed(x)   # (B, D, H', W')
        B, D, H_, W_ = pe.shape
        pe_flat = pe.flatten(2).transpose(1, 2)   # (B, N, D)
        pe_pool = pe_flat.mean(dim=1)            # (B, D)

    print(f"\n2) Patch-embed shape: {pe.shape} -> flattened {pe_flat.shape}")
    m = pe_pool.mean(dim=1).cpu().numpy()
    s = pe_pool.std(dim=1).cpu().numpy()
    print("   Patch-embed pooled mean per sample:", np.round(m, 6))
    print("   Patch-embed pooled std  per sample:", np.round(s, 6))
    identical_pe = torch.allclose(pe_pool, pe_pool[0].unsqueeze(0).expand_as(pe_pool))
    print("   Patch-embed pooled identical across batch?:", bool(identical_pe))

    # 3) full backbone forward -> pooled features
    with torch.no_grad():
        pooled = backbone.forward_features(x, return_pooled=True, apply_mask=False)  # (B, D)
    print(f"\n3) Backbone pooled features shape: {pooled.shape}")
    pooled_means = pooled.mean(dim=1).cpu().numpy()
    pooled_stds = pooled.std(dim=1).cpu().numpy()
    print("   Pooled mean per sample:", np.round(pooled_means, 6))
    print("   Pooled std  per sample:", np.round(pooled_stds, 6))
    identical_pooled = torch.allclose(pooled, pooled[0].unsqueeze(0).expand_as(pooled))
    print("   Pooled features identical across batch?:", bool(identical_pooled))

    # 4) cosine similarities between samples
    if pooled.shape[0] >= 2:
        f0 = pooled[0].cpu().numpy()
        for i in range(1, min(5, pooled.shape[0])):
            fi = pooled[i].cpu().numpy()
            cos = np.dot(f0, fi) / (np.linalg.norm(f0) * np.linalg.norm(fi) + 1e-12)
            print(f"   Cosine pooled[0] vs pooled[{i}]: {cos:.6f}")

    # 5) NaN / Inf checks
    has_nan = torch.isnan(pooled).any().item()
    has_inf = torch.isinf(pooled).any().item()
    print("\n5) pooled has NaN?", has_nan, " pooled has Inf?", has_inf)

    # 6) first VimBlock activation stats
    first_block = None
    if hasattr(backbone, "backbone") and len(backbone.backbone) > 0:
        first_block = backbone.backbone[0]
    elif isinstance(backbone, nn.Sequential) and len(backbone) > 0:
        first_block = backbone[0]

    if first_block is not None:
        with torch.no_grad():
            first_out = first_block(pe_flat)  # (B, N, E)
        print("\n6) First block output shape:", first_out.shape)
        mean_per_sample = first_out.mean(dim=(1,2)).cpu().numpy()
        std_per_sample = first_out.std(dim=(1,2)).cpu().numpy()
        print("   mean per sample:", np.round(mean_per_sample, 6))
        print("   std  per sample:", np.round(std_per_sample, 6))
        identical_first = torch.allclose(first_out, first_out[0].unsqueeze(0).expand_as(first_out))
        print("   First block output identical across batch?:", bool(identical_first))

        # 7) parameter sanity for first block (print a few params)
        print("\n7) First block parameter stats (sample):")
        found = False
        for n, p in first_block.named_parameters():
            found = True
            pv = p.detach().cpu()
            print(f"   {n:30s} | mean={pv.mean():.6f} std={pv.std():.6f} min={pv.min():.6f} max={pv.max():.6f}")
            # print only a few parameters
            break
        if not found:
            print("   (no params found in first block)")

        # 8) small gradient check: single forward/backward through first block
        device = next(first_block.parameters()).device if any(p.requires_grad for p in first_block.parameters()) else x.device
        x_rand = torch.randn(2, pe_flat.shape[1], pe_flat.shape[2], device=device, requires_grad=True)
        try:
            out = first_block(x_rand).sum()
            out.backward()
            grad_std = x_rand.grad.std().item() if x_rand.grad is not None else float("nan")
            print("\n8) Grad check: input grad std:", grad_std)
        except Exception as e:
            print("\n8) Grad check failed:", e)
    else:
        print("\nNo first block found for inspection (structure differs).")

    # 9) patch_embed parameter stats
    print("\n9) patch_embed parameters:")
    pe_w = backbone.patch_embed.weight.detach().cpu()
    print("   patch_embed.weight mean:", float(pe_w.mean()), " std:", float(pe_w.std()))
    if backbone.patch_embed.bias is not None:
        print("   patch_embed.bias mean:", float(backbone.patch_embed.bias.detach().cpu().mean()))
    print("   patch_embed.weight any NaN?", bool(torch.isnan(pe_w).any()))
    if backbone.patch_embed.bias is not None:
        print("   patch_embed.bias any NaN?", bool(torch.isnan(backbone.patch_embed.bias.detach().cpu()).any()))

    print("\n=== DIAGNOSTICS COMPLETE ===\n")

def main():
    # build dataset + loader
    tfm = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
    ])
    ds = IDRiDDataset(root=IDRID_PATH, split="test", transform=tfm)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    # load model pieces
    assert os.path.exists(CHECKPOINT_PATH), f"Checkpoint not found: {CHECKPOINT_PATH}"
    backbone, head = load_backbone_and_head(CHECKPOINT_PATH)

    # get first batch
    batch = next(iter(loader))
    diagnostics(backbone, head, batch)

if __name__ == "__main__":
    main()

Found 103 images for split=test


AttributeError: module 'config' has no attribute 'VMAMBA_EMBED_DIM'